In [ ]:
!pip install -q bitsandbytes accelerate peft trl

In [ ]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
model_id = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)


In [ ]:
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
# === 指令模板 ===
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a local resident of a region around the world. Answer the following common-sense question based on real-world knowledge in that region. Respond in clear, standard English. Be concise and factual. Provide only the essential answer without any explanation, introduction, or punctuation.

### Input:
{}

### Response:
{}"""

prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are a local resident of a region around the world. Answer the following common-sense question based on real-world knowledge in that region. Respond in clear, standard English. Be concise and factual. Provide only the essential answer without any explanation, introduction, or punctuation.

### Input:
{}

### Response:"""


In [ ]:
EOS_TOKEN = tokenizer.eos_token
# === 数据格式化函数 ===
def formatting_prompts_func(examples):
    questions = examples["question"]
    answers = examples["answer"]
    outputs = []
    for q, a in zip(questions, answers):
        text = alpaca_prompt.format(q, a) + EOS_TOKEN
        outputs.append(text)
    return {"text": outputs}

# === 加载训练数据 ===
build_data = pd.read_csv("/kaggle/input/train-data/build_data.csv",header=0, delimiter="\t", quoting=3)
test_df = pd.read_csv("/kaggle/input/train-data/output_with_translation.tsv", header=0, delimiter="\t", quoting=3)
train_df, val_df = train_test_split(build_data, test_size=0.2, random_state=42)

train_dict = {'answer': train_df["answer_en"], 'question': train_df['question_en']}
val_dict = {'answer': val_df["answer_en"], 'question': val_df['question_en']}
test_dict = {"question": test_df['question_en']}

train_dataset = Dataset.from_dict(train_dict)
val_dataset = Dataset.from_dict(val_dict)
test_dataset = Dataset.from_dict(test_dict)

# 格式化
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
val_dataset = val_dataset.map(formatting_prompts_func, batched=True)

In [ ]:
print(train_dataset[0]["text"])

In [ ]:
os.makedirs('/kaggle/working/checkpoint', exist_ok=True)
training_args = TrainingArguments(
    output_dir="/kaggle/working/checkpoint",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    seed=3407,
    report_to="none",
)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:

trainer.train()

In [ ]:
model.save_pretrained("/kaggle/working/final_lora")
tokenizer.save_pretrained("/kaggle/working/final_lora")

In [ ]:
from tqdm import tqdm

model.half()
# === 推理 ===
test_df = pd.read_csv("/kaggle/input/train-data/output_with_translation.tsv", sep='\t')

test_questions = test_df["question_en"].tolist()
test_ids = test_df["id"].tolist()

predictions = []

for q in tqdm(test_questions, desc="Inference"):
    prompt = alpaca_prompt.format(q, "") 
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024,
        padding=False,
        add_special_tokens=True,
    ).to(model.device)
    
    input_length = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True,
        )

    generated_tokens = outputs[0][input_length:]
    answer = tokenizer.decode(
        generated_tokens,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    ).strip()
    
    predictions.append(answer)


In [ ]:
# === 保存结果 ===
result_df = pd.DataFrame({
    "id": test_ids,
    "answer_en": predictions
})

result_df.to_csv("/kaggle/working/saq_instruction_tuning.csv", index=False, sep='\t')